In [1]:
include("JQlaw.jl")
using .JQlaw
using Plotly

WebIO._IJuliaInit()

## Simple Orbit Raise Example

In [2]:
orbit_inital_keplarian = KeplarianOrbit(
                450000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

orbit_target_keplarian = KeplarianOrbit(
                600000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.5,
                0,
                10, 
                500.,
                10^6,
                10^3)

Sat_Params = sat_params(100, 0.8, 0.5/100.)

time_initial = 59715.0 # MJD

# convert keplarian orbits into equinoctial orbits, store orbits as arrays
orbit_initial_equinoctial = Keplarian2Equinoctial(orbit_inital_keplarian)
orbit_target_equinoctial = Keplarian2Equinoctial(orbit_target_keplarian)

orbit = [orbit_initial_equinoctial.p,
        orbit_initial_equinoctial.f,
        orbit_initial_equinoctial.g,
        orbit_initial_equinoctial.h,
        orbit_initial_equinoctial.k,
        orbit_initial_equinoctial.L]

orbit_target = [orbit_target_equinoctial.p,
        orbit_target_equinoctial.f,
        orbit_target_equinoctial.g,
        orbit_target_equinoctial.h,
        orbit_target_equinoctial.k,
        orbit_target_equinoctial.L]

# define histories to save outcomes to
Q_hist = []
orbit_hist = []
time_hist = []

#initialize_values
step_count = 0
Q_value = Q(orbit, orbit_target, Q_Params, Sat_Params)
time = time_initial
# add initial values to history 
push!(Q_hist, Q_value)
push!(orbit_hist, orbit)
push!(time_hist, time)

# integrate until hits max number of steps or Q convergence 
while (step_count < Q_Params.max_iter) && (Q_value > Q_Params.Q_convergence)
    thrust, F_t, F_r, F_n = evaluate_orbit_location(orbit, orbit_target, Q_Params, Sat_Params)
    if thrust
        GVE(orbit, time) = GaussVariationalEquationsEquinoctial(orbit, time; accel=[F_r,F_t,F_n])
        orbit_new, error = RK45(orbit, time, Q_Params.step_size, dOEdt = GVE)
        Q_new = Q(orbit_new, orbit_target, Q_Params, Sat_Params)
        if Q_new < Q_value
            Q_value = Q_new
            orbit = orbit_new
        else
            orbit, error = RK45(orbit, time, Q_Params.step_size, dOEdt = GaussVariationalEquationsEquinoctial)
            Q_value = Q(orbit, orbit_target, Q_Params, Sat_Params)
        end
    else
        orbit, error = RK45(orbit, time, Q_Params.step_size, dOEdt = GaussVariationalEquationsEquinoctial)
        Q_value = Q(orbit, orbit_target, Q_Params, Sat_Params)
    end
    time = time + (Q_Params.step_size / 60. / 60. / 24.)
    push!(Q_hist, Q_value)
    push!(orbit_hist, orbit)
    push!(time_hist, time)
    print(Q_value, "\n")
end

print(Q_hist)


# # Initialize history to store propogated orbits and error bounds
# orbit_history = [orbit_initial]
# error_history = [[0.,0.,0.,0.,0.,0.]]
# time_history = [time_initial]

# # Define integrator
# GVE_Spiral(orbit, time) = GaussVariationalEquationsEquinoctial(orbit, time; accel=[0.,1.,0.])

# # Integrate orbit with no external forces
# time = time_initial
# print("Starting integration...\n")
# while time < time_end
#     global time = time_history[end]
#     orbit = orbit_history[end]
#     orbit_new, error = RK45(orbit, time, step_size, dOEdt = GVE_Spiral)
#     push!(time_history, time + (step_size / 60. / 60. / 24.))
#     push!(orbit_history, orbit_new)
#     push!(error_history, error)
# end
# print("Integration finished!\n")

# # store ECI terms
# ECI_x = []
# ECI_y = []
# ECI_z = []
# for orbit in orbit_history
#     equinoctial_orbit = EquinoctialOrbit(orbit[1], orbit[2], orbit[3], orbit[4], orbit[5], orbit[6])
#     ECI_orbit = Equinoctial2ECI(equinoctial_orbit)
#     push!(ECI_x, ECI_orbit[1])
#     push!(ECI_y, ECI_orbit[2])
#     push!(ECI_z, ECI_orbit[3])
# end

5.236278816394392e8
4.9129088147076845e8
4.632591877056975e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.452469279371275e8
4.2890594618124384e8
4.010049728115263e8
3.721243415439715e8
3.451618093167987e8
3.2311813206295514e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
3.1520584761078733e8
2.955420846264036e8
2.7159092926174015e8
2.481395947900941e8
2.2722551148817983e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.1300469618848336e8
2.0336768140770242e8
1.850331583078985e8
1.6591245497602487e8
1.4815484582072031e8
1.3369690552500097e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.2860389657520905e8
1.166883079127281e8
1.0213019730868906e8
8.813016354277432e7
7.5918379